# 📊 Feature Group Detailed Analysis
## Hull Tactical Market Prediction - Phase 2

**Date:** 2025-11-11

**Objectives:**
1. 모든 Feature 그룹별 시계열 분석 (E, V, M, I, P, S, D)
2. 각 Feature의 통계적 특성 파악
3. Feature 간 시간에 따른 패턴 비교
4. 이상치 및 트렌드 탐지
5. 그룹별 비교 분석

## 1. Setup and Imports

In [ ]:
# Standard libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Get project root
notebook_dir = os.getcwd() if '__file__' not in dir() else os.path.dirname(os.path.abspath(__file__))
project_root = os.path.dirname(notebook_dir) if 'notebooks' in notebook_dir else notebook_dir
sys.path.insert(0, project_root)

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy import stats

# Project utilities
from src.data import DataLoader
from src.utils import set_seed, load_config

# Load configuration
config_path = os.path.join(project_root, 'conf', 'params.yaml')
config = load_config(config_path)
set_seed(config['seed'])

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['figure.dpi'] = 100

print('✅ Setup complete!')
print(f'📋 Seed: {config["seed"]}')

## 2. Load Data

In [ ]:
# Change to project root
os.chdir(project_root)
print(f"📂 Working directory: {os.getcwd()}")

# Initialize DataLoader
loader = DataLoader()

# Load data
train_df, test_df = loader.load_data()

print(f'\n📦 Data Shape:')
print(f'  Train: {train_df.shape}')
print(f'  Test: {test_df.shape}')
print(f'\n📅 Date range: {train_df["date_id"].min()} - {train_df["date_id"].max()}')

# Display feature groups
print('\n🏷️ Feature Groups:')
for group, features in loader.feature_groups.items():
    print(f'  {group}: {len(features)} features')

train_df.head()

In [ ]:
# Helper functions: plotting and statistics

def plot_feature_timeseries(df, features, group_name, max_cols=4):
    """Plot time series for all features in a group.

    df : DataFrame with 'date_id' and features
    features : list of feature names
    group_name : string
    """
    if not features:
        print(f"No features found for {group_name} group")
        return

    n_features = len(features)
    n_cols = min(max_cols, n_features)
    n_rows = (n_features + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
    if n_rows * n_cols == 1:
        axes = np.array([axes])
    axes = np.array(axes).reshape(-1)

    for idx, feat in enumerate(features):
        ax = axes[idx]
        ax.plot(df['date_id'], df[feat], linewidth=0.8, alpha=0.8)
        ax.set_title(f'{feat}', fontsize=10, fontweight='bold')
        ax.set_xlabel('Date ID', fontsize=9)
        ax.set_ylabel('Value', fontsize=9)
        ax.grid(True, alpha=0.3)

        # Add small stat box
        mean_val = df[feat].mean()
        std_val = df[feat].std()
        stats_text = f'Mean: {mean_val:.4f}\nStd: {std_val:.4f}'
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
                verticalalignment='top', fontsize=8,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # Hide unused axes
    for j in range(n_features, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f'{group_name} Group - Time Series ({n_features} features)', fontsize=14)
    plt.tight_layout()
    plt.show()


def get_feature_statistics(df, features, group_name):
    """Return DataFrame with descriptive stats and moments for features."""
    from scipy import stats as scipy_stats
    if not features:
        return pd.DataFrame()

    stats_rows = []
    for feat in features:
        s = df[feat]
        valid = s.dropna()
        row = {
            'Feature': feat,
            'Group': group_name,
            'Count': int(valid.shape[0]),
            'Missing': int(s.isna().sum()),
            'Missing %': float(s.isna().sum() / max(1, len(s)) * 100),
            'Mean': float(valid.mean()) if len(valid) else np.nan,
            'Std': float(valid.std()) if len(valid) else np.nan,
            'Min': float(valid.min()) if len(valid) else np.nan,
            'Q25': float(valid.quantile(0.25)) if len(valid) else np.nan,
            'Median': float(valid.median()) if len(valid) else np.nan,
            'Q75': float(valid.quantile(0.75)) if len(valid) else np.nan,
            'Max': float(valid.max()) if len(valid) else np.nan,
            'Skewness': float(scipy_stats.skew(valid)) if len(valid) else np.nan,
            'Kurtosis': float(scipy_stats.kurtosis(valid)) if len(valid) else np.nan
        }
        stats_rows.append(row)

    return pd.DataFrame(stats_rows)


def plot_distribution_comparison(df, features, group_name, max_features=12):
    """Plot histograms for first max_features in the list."""
    if not features:
        return
    plot_feats = features[:max_features]
    n = len(plot_feats)
    cols = 4
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, 3 * rows))
    axes = np.array(axes).reshape(-1)

    for i, feat in enumerate(plot_feats):
        ax = axes[i]
        df[feat].dropna().hist(bins=50, ax=ax, alpha=0.7, edgecolor='black')
        ax.set_title(feat, fontsize=10)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        ax.grid(True, alpha=0.3)

    for j in range(n, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f'{group_name} Group - Distribution (showing first {n} features)')
    plt.tight_layout()
    plt.show()

print('✅ Helper functions inserted')

In [ ]:
# E Group analysis (Economic indicators)
e_features = loader.feature_groups.get('E', [])
print(f"📊 Analyzing E Group ({len(e_features)} features)\n")

# Time series (date_id vs value)
plot_feature_timeseries(train_df, e_features, 'E')

# Distribution (first 12 features)
plot_distribution_comparison(train_df, e_features, 'E')

# Statistics table
e_stats = get_feature_statistics(train_df, e_features, 'E')
print('\n📈 E Group Statistical Summary:')
print('='*80)
display(e_stats)

In [ ]:
# V Group analysis (Volatility indicators)
v_features = loader.feature_groups.get('V', [])
print(f"📊 Analyzing V Group ({len(v_features)} features)\n")

plot_feature_timeseries(train_df, v_features, 'V')
plot_distribution_comparison(train_df, v_features, 'V')

v_stats = get_feature_statistics(train_df, v_features, 'V')
print('\n📈 V Group Statistical Summary:')
print('='*80)
display(v_stats)

In [ ]:
# M Group analysis (Market indicators)
m_features = loader.feature_groups.get('M', [])
print(f"📊 Analyzing M Group ({len(m_features)} features)\n")

plot_feature_timeseries(train_df, m_features, 'M')
plot_distribution_comparison(train_df, m_features, 'M')

m_stats = get_feature_statistics(train_df, m_features, 'M')
print('\n📈 M Group Statistical Summary:')
print('='*80)
display(m_stats)

In [ ]:
# P, S, D group analyses + cross-group aggregation, CV/missing analysis, and exports
import os
from IPython.display import display

results_dir = os.path.join(os.getcwd(), 'results', 'feature_analysis')
os.makedirs(results_dir, exist_ok=True)

group_vars = {}
for grp in ['P', 'S', 'D']:
    feats = loader.feature_groups.get(grp, [])
    print(f"\n📊 Analyzing {grp} Group ({len(feats)} features)\n")
    if len(feats) == 0:
        print(f" - No features found for group {grp}, skipping.")
        continue

    # Time-series and distribution plots
    try:
        plot_feature_timeseries(train_df, feats, grp)
        plot_distribution_comparison(train_df, feats, grp)
    except Exception as e:
        print(f"Warning: plotting for group {grp} failed: {e}")

    # Stats
    stats = get_feature_statistics(train_df, feats, grp)
    group_vars[f'{grp.lower()}_stats'] = stats
    print('\n📈 {grp} Group Statistical Summary:'.format(grp=grp))
    display(stats)

# Collect any existing per-group stats (e, v, m, i, p, s, d)
parts = []
for name in ['e_stats','v_stats','m_stats','i_stats','p_stats','s_stats','d_stats']:
    if name in globals():
        parts.append(globals()[name])

if len(parts) == 0:
    print('No group stats found to aggregate. Make sure group cells have been run at least once.')
else:
    all_stats = pd.concat(parts, axis=0)
    # Ensure numeric columns
    for col in ['Mean','Std']:
        if col not in all_stats.columns:
            all_stats[col] = np.nan

    # Coefficient of Variation (cv)
    all_stats['cv'] = all_stats.apply(lambda r: (r['Std'] / abs(r['Mean'])) if pd.notnull(r['Mean']) and abs(r['Mean'])>1e-9 else np.nan, axis=1)

    # Group-level summary
    group_summary = all_stats.groupby('Group').agg(
        features=('Feature', 'count'),
        missing_total=('Missing', 'sum'),
        mean_of_means=('Mean', 'mean'),
        mean_of_std=('Std', 'mean'),
        mean_of_cv=('cv', 'mean'),
        mean_of_skew=('Skewness', 'mean'),
        mean_of_kurtosis=('Kurtosis', 'mean')
    ).reset_index()

    # Missing-data ranking
    missing_by_feature = all_stats[['Group','Missing']].sort_values('Missing', ascending=False)

    # Save CSVs
    all_stats_path = os.path.join(results_dir, 'all_feature_stats.csv')
    group_summary_path = os.path.join(results_dir, 'group_summary.csv')
    missing_path = os.path.join(results_dir, 'missing_by_feature.csv')

    all_stats.to_csv(all_stats_path)
    group_summary.to_csv(group_summary_path, index=False)
    missing_by_feature.to_csv(missing_path, index=False)

    print('\n✅ Aggregation complete — saved CSVs to:')
    print(' -', all_stats_path)
    print(' -', group_summary_path)
    print(' -', missing_path)

    display(group_summary)
    print('\nTop 20 features by missing values:')
    display(missing_by_feature.head(20))

print('\nAll done. Run the notebook cells (or this final cell) in order to generate plots and exports.')